#Тема 21. Механизм внимания. Сети Transformer, BERT



Имея векторные представления для слов можем проводить дальнейшую обработку с этими векторами. Представив текст как последовательность слов логично использовать рекуррентные нейронные сети для такой обработки. Например, с помощью LSTM сетей можем пытаться предсказать следующее слово по уже имеющимся в предложении. 

Но для задач обработки именно текстов возникают проблемы.

В сетях типа LSTM влияние разных элементов последовательности друг на друга устанавливается косвенно, через влияние на соседние элементы. К тому же такое влияние идет только в одну строну, предыдущие элементы влияют на последующие через изменение состояния нейрона, но не наоборот. 

Частично это исправляется в *двунаправленных сетях*, где последовательность читается и слева направо и справа налево, а результаты объединяются. 

![img](https://www.tutorialexample.com/wp-content/uploads/2020/07/The-structure-of-BiLSTM.png)

Но даже этого оказывается мало для качественной обработки текстов. В текстах разные слова, даже находящиеся далеко от текущего, могут сильно влиять на него. 

Нужно разрешить элементам последовательности влиять друг на друга произвольно и обучать такие зависимости. 

Так работает **"механизм внимания"** (attention mechanism). Дальше мы будем подразумевать последовательности слов, но вообще, все изложенное применимо к любым последовательностям, например, последовательностям звуков - речи.

# Механизм внимания
Давайте каждому элементу входной последовательности (это вектор: или векторные представления входной информации или выход предыдущего слоя) сопоставим три вектора: Ключ (Key) К, Запрос (Query) Q, Значение (Value) V. Их можно получить умножением вектора на некоторую матрицу, размер которой мы выбираем сами. Такие матрицы будут обучаться.

![img](https://jalammar.github.io/images/t/self-attention-matrix-calculation.png) 


Для Запроса Q конкретного элемента (на картинке первого) посчитаем его скалярные произведения  с *каждым* Ключом K всех других элементов (включая себя), получим набор чисел score, поместим их в вектор и с помощью softmax переведем его в вид **коэффициентов внимания** (в диапазоне от 0 до 1). 

Такие коэффициенты показывают, насколько похожи (т.е. насколько связаны между собой) этот Запрос Q на все Ключи K. 

![img](https://miro.medium.com/max/3789/1*jf__2D8RNCzefwS0TP1Kyg.gif)



Далее каждое Значение V умножим на соответствующий коэффициент внимания score и просуммируем результаты для всех Значений. 

Так мы создадим новый вектор output для текущего Запроса Q. Например, если все другие слова (вектора Значений V) в последовательности непохожи на слово (вектор) Запроса Q, их коэффициент внимания будет 0, и на выходе останется только Значение V от того же Запроса Q.  

![img](https://miro.medium.com/max/3789/1*1je5TwhVAwwnIeDFvww3ew.gif)



Сделаем так для всех Запросов Q от разных элементов входной поледовательности.

В результате получим выходную последовательность, в каждом элементе которой явно учтены влияния всех других элементов входной последовательности.


![img](https://miro.medium.com/max/3156/1*_92bnsMJy8Bl539G4v93yg.gif)



Все описанные операции выполняются в специальном слое **внимания**, который реализован как слой нейронной сети. Принимает последовательность векторов,  возвращает другую последовательность векторов (возможно другой размерности), в которой учтено влияние входов друг на друга.   

Матрицы Запроса, Ключа, Значения будут обучаться, поэтому компьютер сможет сам подобрать их значения. Здесь все операции дифференцируемые, поэтому обучение ведется известным нам методом обратного распространения ошибки. 


## Множественное внимание
Для еще лучшей работы используют множественное внимание, когда одновременно применяют несколько разных матриц Q,K,V, а результаты потом объединяют другой обучаемой матрицей.

![img](https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)

# Сеть Transformer
На основе механизма внимания работает сеть Transformer.

![img](https://miro.medium.com/max/3349/1*BHzGVskWGS_3jEcYYi6miQ.png)

Это довольно сложная нейронная сеть, состоит из двух частей, кодера и декодера.

**Кодер** состоит из нескольких блоков. Первый блок  принимает входную последовательность, остальные блоки - выходы предыдущих блоков. Каждый блок применяет механизм множественного внимания, а также простую полносвязную сеть. Внутри блока используют перекрестные связи, по аналогии с ResNet блоками, и нормализацию.

**Декодер** также состоит из блоков и похож на кодер, но (его первый блок) принимает *выходную* последовательность, а также выходы из соответствующего блока кодера. 

Выходы декодера обрабатываются линейным слоем и пропускаются через softmax, который возвращает уровень уверенности в следующем слове (токене) выходной последовательности.

Поскольку это не рекуррентная сеть, то необходим дополнительный способ для кодирования времени, что выполняется в слое Positional Encoding добавлением к векторным представлениям входов вектора, отвечающего за время (какая-то последовательность, например, по функции косинуса). 

В процессе обучения входная и выходная последовательности известны. Например, для задачи перевода текста это предложения (последовательности слов) на одном и другом языке. Поэтому известны цели, чему обучать сеть - уровень уверенности должен быть максимальным для того слова, которое имеется в выходной последовательности. Сеть обучается на множестве примеров входных и выходных последовательностей.   

В процессе предсказания выходная последовательность не известна, поэтому строится итерационно:
- сначала используется пустая выходная последовательность с токеном начала последовательности и предсказывается (выход softmax) следующий ее токен.
- этот предсказанный токен вставляется в последовательность и предсказывается следующий, и т.д., пока не будет предсказан токен конца последовательности.

В таком подходе предсказанная выходная последовательность может быть другой длины. 


# BERT
На основе сетей Transformer работает, пожалуй, одна из наиболее успешных сетей для работы с текcтом - сеть BERT (Bidirectional Encoder Representations from Transformers).

![img](https://yashuseth.files.wordpress.com/2019/06/fig9.png?w=412&h=375)



Эта сеть использует только **кодер** из Transformer который принимая входную последовательность токенов (слов) возвращает измененную последовательность (которая потом поступила бы на декодер). 

Изменили цель обучения такой сети. В обучении BERT используется два подхода:



## 1) Предсказание маски. Masked Language Modelling
Некоторое количество токенов (15%) в последовательности заменяется специальным токеном [MASK] и ставится задача правильно предсказать такие токены. Для этого кодер дополняется слоями классификации (линейный слой, активация, нормализация), проекции в пространство словаря и softmax, а в функции ошибки учитываются только выходы для токенов помеченных токеном [MASK].

![img](https://miro.medium.com/max/3789/0*ViwaI3Vvbnd-CJSQ.png)



## 2) Предсказание следующей последовательности. Next Sentence Prediction
В качестве входа подаются *две последовательности*, одна за другой и сеть учится предсказывать, что вторая последовательность является логическим продолжением первой. 

Для этого при обучении создаются примеры как "правильных" последовательностей взятых подряд из обучающих данных (предложения из текста, взятые подряд), так и "неправильных", взятых в случайном порядке. Последовательности разделяются токеном [SEP], первая последовательность начинается с токена [CLS].

Выход кодера (конкретно для входного [CLS])  классифицируется (добавлением одного слоя классификации с softmax) на два класса: последовательности действительно логически продолжают друг друга или нет.



Обучение ведется с комбинированной функцией ошибки обоих подходов.

Чтобы можно было различить последовательности их кодируют специальным образом:

- к векторным представлениям токенов первой последовательности добавляется специальная последовательность A, к токенам второй - B (эти специальные последовательности были предобучены)
- и, как и в Transformer, добавляется последовательность для кодирования времени. 

![img](https://miro.medium.com/max/3789/0*m_kXt3uqZH9e7H4w.png)







Обучая такую сеть на *огромном* объеме текстов получили одну из самых мощных на сегодня сетей для работы с текстом. 

Обратите внимание, что кодер Transformer переводит последовательность в другое векторное представление. Это свойство нам еще пригодится. 

Посмотрим пример. 

# Пример работы BERT

Решается задача определения отзыва на фильм как положительного или отрицательного. 

Используется DistilBERT (упрощенная версия BERT) для перевода отзыва в векторное пространство (размерностью 768), а затем  логистическая регрессия для классификации.

Сети, основанные на transformer реализованы в библиотеке [transformers](https://huggingface.co/transformers/).

<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Набор данных
Используем набор данных [SST2](https://nlp.stanford.edu/sentiment/index.html), пример ниже (1 - положительные, 0 - отрицательные).

<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>



In [1]:
# устанавливаем библиотеку 
!pip install transformers

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 901kB 24.5MB/s 
     |████████████████████████████████| 3.3MB 45.0MB/s 


In [2]:
# подключаем библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

Загружаем набор данных


In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

Оставим только 2,000 штук для скорости.

In [4]:
batch_1 = df[:2000]

Сколько положительных и отрицательных?

In [5]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

### Загружаем обученную модель BERT


In [6]:
#  DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Хотите BERT вместо distilBERT? Раскоментируйте
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# загружаем
tokenizer = tokenizer_class.from_pretrained(pretrained_weights) # токенизатор для разделения на слова
model = model_class.from_pretrained(pretrained_weights) # модель 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Надо сделать некоторую предобработку данных.

### Токенизация
Разбиваем предложения на токены в подходящем формате:
- разбиваем отзыв на отдельные токены (слова, части слов и т.п.)
- добавляем токены CLS и SEP
- заменяем токены на их номер ID



In [7]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Набивка
Наиболее быстрая работа когда все предложения в одном массиве, для чего их надо дополнить до одинакового размера.

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [9]:
np.array(padded).shape

(2000, 59)

Надо указать, что добавленные в набивке нулевые значения нужно игнорировать. 

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

## Запускаем модель

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

Команда`model()` запускает расчет модели BERT, результат вернется в `last_hidden_states`.

In [11]:
input_ids = torch.tensor(padded)  # тензоры
attention_mask = torch.tensor(attention_mask) # тензоры

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask) # запускаем

При классификации добавляется токен `[CLS]` в начале каждого предложения, выход для него и есть векторное представление для всего предложения.  

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />



In [12]:
features = last_hidden_states[0][:,0,:].numpy() # это векторное представление отзывов

Метки классов:

In [13]:
labels = batch_1[1] # метки классов

In [14]:
# разделяем данные на обучающие и тестовые
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />


### Классификация
После применения BERT имеем некоторые вектора для каждого отзыва. Создаем и обучаем логистическую регресию.

In [15]:
lr_clf = LogisticRegression() # создаем
lr_clf.fit(train_features, train_labels) # обучаем

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
# проверяем, как работает
lr_clf.score(test_features, test_labels)

0.866

Получили 86% аккуратности предсказания на тестовых примерах. Это и не самая плохая и не самая хорошая классификация. 

Здесь можно посмотреть лучшие из [достигнутых результатов](http://nlpprogress.com/english/sentiment_analysis.html) для этого набора данных (**96.8**).


#Задания
Попробуйте самостоятельно сделать классификацию написанного вами отзыва на фильм (на англ. языке) используя обученные здесь модели.

(нужно повторить процесс: токенизация, набивка, расчет BERT и получение вектора для отзыва, его классификация)

# Ссылки

Использованы и адаптированы материалы:

Механизм внимания:
https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a#8481

Это самое лучшее объяснение работы Transformer:
https://jalammar.github.io/illustrated-transformer/

https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04 

BERT 

https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270

http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

https://yashuseth.blog/2019/06/12/bert-explained-faqs-understand-bert-working/
